# Data Preprocessing

## Data Cleaning

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',30)

In [2]:
df = pd.read_excel('Saha_et_al_2020_ERL_Data.xlsx', sheet_name='Data')
df.head()

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,PP2,PP7,AirT,DAF_TD,DAF_SD,WFPS25cm,NH4,NO3,Clay,Sand,SOM
0,2012-02-09,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,0.00,0.00,-2.0,276,241,0.666508,11.046340,22.940812,62.5,637.5,1.174072
1,2012-02-10,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,0.00,0.00,-2.4,277,242,0.640608,11.008087,22.959578,62.5,637.5,1.174072
2,2012-02-18,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,3.30,8.64,0.3,285,250,0.728085,10.831669,23.221928,62.5,637.5,1.174072
3,2012-02-19,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,0.00,8.13,-3.8,286,251,0.686872,10.849792,23.271978,62.5,637.5,1.174072
4,2012-03-16,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,1.02,8.39,17.6,312,277,0.716221,10.204748,24.206855,62.5,637.5,1.174072


### Feature Transformation Pipeline

In [210]:
class DataCleaner:
    """
        Class for cleaning data.
        
        Callable Methods:
        ----------------
        clean_data: To clean up raw input data with features as on training data
        clean_data_partial: To clean up customized input data
        inverse_target_transformer: To inverse transform predicted value
    """
    def __init__(self, df):
        self.df = df

    def column_modifier(X):
        """
         Function for modifying dataset columns.
        """
        # Creating 'Mean_DAF'
        try:
            X['Mean_DAF'] = (X['DAF_SD']+X['DAF_TD'])*0.5
        except Exception as e:
            print("Couldn't create 'Mean_DAF': ", e)

        # Dropping columns
        try:
            X = X.drop(columns=['Date', 
                                'Year', 
                                'Experiment', 
                                'DataUse', 
                                'Replication', 
                                'VegType', 
                                'DAF_SD', 
                                'DAF_TD'], axis=1)
        except Exception as e:
            print("Couldn't drop features: ", e)

        return X

    def data_modifier(X):
        """
         Function for basic data cleaning inlcuding:
        """
        # Modifying 'N2O'
        try:
            X['N2O'] = X['N2O'] + np.abs(np.min(X['N2O']))
        except Exception as e:
            print("Couldn't transform 'N2O': ", e)

        # Converting datatypes of numerical features to float
        try:
            num_feat = [col for col in X  if X[col].dtypes in ['int64', 'float64'] and X[col].nunique()>20]

            for feat in num_feat:
                X[feat] = X[feat].astype('float')
        except Exception as e:
            print("Couldn't convert numerical feature datatypes: ", e)

        # Converting datatypes of categorical and discrete numerical features
        try:
            cat_feat = [col for col in X if col not in num_feat]

            to_cat = [feat for feat in cat_feat if X[feat].dtypes!='int'] # List of features to be converted to type 'category'
            to_int = [feat for feat in cat_feat if feat not in to_cat] # List of features to be converted to type 'int'

            for feat in to_cat:
                    X[feat] = X[feat].astype('category')
            for feat in to_int:
                    X[feat] = X[feat].astype('int')
        except Exception as e:
            print("Couldn't convert categorical feature datatypes: ", e)

        return X

    def missing_imputer(X):
        """
            Function for missing value imputation
        """
        try:
            X['WFPS25cm'] = X['WFPS25cm'].fillna(0.321753) # constant value imputation
            X['NH4'] = X['NH4'].fillna(np.mean(X['NH4'])) # mean imputation
            X['NO3'] = X['NO3'].fillna(np.mean(X['NO3'])) # mean imputation
        except Exception as e:
            print("Failed to impute missing values: ", e)
        return X

    def numerical_transformer(X):
        """
            Function for transforming numerical features.
        """
        try:
            num_feat = [col for col in X if X[col].dtypes in ['int64', 'float64'] and X[col].nunique()>20]                             
            # filtering out positively skewed features for transformation
            positively_skewed = [feat for feat in num_feat if X[feat].skew()>0.7]

            for feat in positively_skewed:
                # log transformation
                X[feat] = np.log1p(X[feat])
                # Winsorizing
                q1 = X[feat].quantile(0.25)
                q3 = X[feat].quantile(0.75)
                iqr = q3 - q1
                upper = q3+(1.5*iqr)
                lower = q1-(1.5*iqr)
                X[feat] = np.where(X[feat]>upper, upper, np.where(X[feat]<lower, lower, X[feat]))
        except Exception as e:
            print("Couldn't transform numerical features: ", e)

        return X

    def binarize_nrate(X):
        """
            Function for binarizing N_rate values
        """
        if X<170:
            return 0
        else:
            return 1

    def binarize_som(X):
        """
            Function for binarizing SOM values
        """
        if X<2:
            return 0
        else:
            return 1

    def bin_month(X):
        """
            Function for binning month values
        """
        try:
            # dictionary for mapping month to season
            season_map = {'January':'Winter',
                          'February':'Winter',
                          'March':'Spring',
                          'April':'Spring',
                          'May':'Spring',
                          'June':'Summer',
                          'July':'Summer',
                          'August':'Summer',
                          'September':'Fall',
                          'October':'Fall',
                          'November':'Fall',
                          'December':'Winter'}

            # mapping the values
            X['Month'] = X['Month'].map(season_map)

            # renaming month to season
            X = X.rename(columns = {'Month':'Season'})
        except Exception as e:
            print("Failed to transform 'Month': ", e)

        return X

    def bin_clay(X):
        """
            Function defining the criteria for binning Clay values
        """
        if X < 128:
            return 1
        elif (X>128) and (X<=230):
            return 2
        else:
            return 3

    def bin_sand(X):
        """
            Function defining the criteria for binning Sand values
        """
        if X <= 125:
            return 1
        elif (X>125) and (X<=491):
            return 2
        else:
            return 3

    def feature_binarizer(X):
        """
            Function for binarizing fetures.
        """
        try:
            # binarizing Vegetation
            X['Vegetation'] = np.where(X['Vegetation']=='Corn', 'Corn', 'Others')

            # binarizing N_rate
            X['N_rate'] = X['N_rate'].apply(binarize_nrate)

            # binarizing SOM
            X['SOM'] = X['SOM'].apply(binarize_som)

        except Exception as e:
            print("Failed to binarize the features: ", e)

        return X

    def feature_binner(X):
        """
            Function for binning.
        """
        try:
            X['Clay'] = X['Clay'].apply(bin_clay) # binning clay
            X['Sand'] = X['Sand'].apply(bin_sand) # binning sand
        except Exception as e:
            print("Failed to bin the features: ", e)
        return X

    def categorical_transformer(X):
        """
            Function for transforming Categorical features
        """
        try:
            X = (X.
                 pipe(self.bin_month).
                 pipe(self.feature_binarizer).
                 pipe(self.feature_binner))
        except Exception as e:
            print("Failed to transform categorical features: ", e)
        return X

    def one_hot_encoder(X):
        """
            Function for One Hot Encoding categorica features
        """
        X_ohe = pd.DataFrame()

        num_feat = [col for col in X  if X[col].dtypes in ['int64', 'float64'] and X[col].nunique()>20]
        cat_feat = [col for col in X if col not in num_feat]

        for feat in cat_feat:
            # one hot encoding
            X1 = pd.get_dummies(X[feat], prefix=f"{feat}_", drop_first=True)

            # dropping one hot encoded features from main dataframe
            X = X.drop(columns=[feat], axis=1)

            # appending new dataframe to dataframe with one hot encoded features
            X_ohe = pd.concat([X_ohe, X1], axis=1)

        X = pd.concat([X, X_ohe], axis=1)

        return X

    def inverse_target_transformer(self, y):
        """
            Function for re-transforming target variable
        """
        y = np.expm1(y) - np.abs(np.min(self.df['N2O']))

        return y

    def clean_data(self):
            """
                Function for cleaning up input data based on training data format
            """
            try:
                X = self.df.copy()
                X = (X.
                     pipe(column_modifier).
                     pipe(data_modifier).
                     pipe(missing_imputer).
                     pipe(numerical_transformer).
                     pipe(categorical_transformer).
                     pipe(one_hot_encoder))

                return X

            except Exception as e:
                print("Failed to clean up input data using clean_data: ", e)

    def clean_data_partial(self):
            """
                Function for cleaning up customized input data
            """
            try:
                X = self.df.copy()
                X = (X.
                     pipe(data_modifier).
                     pipe(missing_imputer).
                     pipe(numerical_transformer).
                     pipe(categorical_transformer).
                     pipe(one_hot_encoder))

            except Exception as e:
                print("Failed to clean up input data using clean_data_partial: ", e)

            return X

In [211]:
obj = DataCleaner(df)

In [213]:
cleaned_df = obj.clean_data()
cleaned_df.head(3)

,N2O,PP2,PP7,AirT,WFPS25cm,NH4,NO3,Mean_DAF,Season__Spring,Season__Summer,Season__Winter,Vegetation__Others,N_rate__1,Clay__2,Clay__3,Sand__2,Sand__3,SOM__1
0,2.510578,0.000000,0.000000,-2.0,0.666508,2.488761,3.175585,5.558757,0,0,1,0,1,0,0,0,1,0
1,2.361374,0.000000,0.000000,-2.4,0.640608,2.485580,3.176368,5.562603,0,0,1,0,1,0,0,0,1,0
2,2.481391,1.458615,2.265921,0.3,0.728085,2.470780,3.187258,5.592851,0,0,1,0,1,0,0,0,1,0


In [214]:
# Saving cleaned data as csv
cleaned_df.to_csv('cleaned_data.csv', index=False)